Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [4]:
import pandas as pd

In [14]:
#read the saved files

bikes_df = pd.read_csv('bikes.csv')
final_foursquare_df = pd.read_csv('foursquare.csv')

#not using yelp results as they are not as comprehensive in terms of number and scope of venues

In [30]:
# add the 'Bike Stop #' 

bikes_df['Bike Stop #'] = bikes_df.index

# merge the df

merged_df = final_foursquare_df.merge(bikes_df, on='Bike Stop #')
merged_df

,Unnamed: 0_x,Bike Stop #,name_x,distance,venue_type,Unnamed: 0_y,name_y,latitude,longitude,free_bikes,empty_slots
0,0,1,NaN,NaN,[],1,"PG. TORRAS I BAGES, 129",41.443365,2.190630,17,7
1,1,2,NaN,NaN,[],2,"C/ SANT OLEGUER, 2",41.377532,2.170732,13,5
2,2,3,NaN,NaN,[],3,"AV. PARAL.LEL, 194",41.375116,2.152309,2,24
3,3,4,Montjuic (Barcelona),181.0,Art Museum,4,"C/ BILBAO, 174",41.409020,2.195415,23,9
4,4,5,NaN,NaN,[],5,"C/ D'ESCORNALBOU, 51",41.418079,2.176399,22,2
...,...,...,...,...,...,...,...,...,...,...,...
627,627,501,La Fábrica del Sol,190.0,Museum,501,"PG. MARITIM, 23 (HOSPITAL DEL MAR)",41.383365,2.194625,2,24
628,628,502,NaN,NaN,[],502,"C/GAVÀ, 81",41.370372,2.138944,9,7
629,629,503,NaN,NaN,[],503,"PG. MARíTIM, 31 B (ANNEXA A LA 12)",41.383475,2.194735,0,21
630,630,504,NaN,NaN,[],504,AV. LITORAL (PG MARÍTIM DEL PORT OLÍMPIC),41.388913,2.199311,4,23


In [37]:
# saved to csv 

merged_df.to_csv('merged.csv')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

I used a pair plot as a inital point of contact during the EDA process. The resulting pair plot can be viewed in files: images > EDA_pairplot.png. There are obvious strong correlations between 'free_bikes', 'empty_slots' and '% used' since they are all related to one another. This however, is not unhelpful information as it confirms that the data extraction process was successful in this limited capacity. The more interesting of the preliminary observations can be seen with the relationship between 'venues_nearby' with all other metrics. 

This will be at the core of the regression modelling that will be addressed later and may prove some interesting patterns. At a glance there seems to be a slight positive trend between 'free_bikes' + 'venue_nearby' and a slight negative trend between 'empty_slots' + 'venues_nearby'.

This qualitatively makes sense as the dataset was pulled around 11pm Barcelona local time. This relationship may show that during the evening, less bikes are used around these venues. These venues tend to be large with much commercial zoning surrounding them - leading to a general decreased use of the bikes during that hour in those zones.

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [9]:
import sqlite3

In [17]:
conn = sqlite3.connect('your_database.db')
cursor = con.cursor()

In [32]:
#bikes to SQL

bikes_df.to_sql('bikes_table', con, index=False, if_exists='replace')

506

In [19]:
# get rid of list to convert to SQL
#foursquare to SQL

final_foursquare_df['venue_type'] = final_foursquare_df['venue_type'].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
final_foursquare_df.to_sql('final_foursquare_table', conn, index=False, if_exists='replace')

633

In [20]:
# get rid of list to convert to SQL
#merged to SQL

merged_df['venue_type'] = merged_df['venue_type'].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
merged_df.to_sql('merged_table', conn, index=False, if_exists='replace')

632

Look at the data before and after the join to validate your data.

In [24]:
#example queries

query = 'SELECT * FROM bikes_table'  
result = pd.read_sql_query(query, conn)

result

,Unnamed: 0,name,latitude,longitude,free_bikes,empty_slots,Bike Stop #
0,0,"RAMBLA DE L'ONZE DE SETEMBRE, 69",41.430000,2.190246,24,1,0
1,1,"PG. TORRAS I BAGES, 129",41.443365,2.190630,17,7,1
2,2,"C/ SANT OLEGUER, 2",41.377532,2.170732,13,5,2
3,3,"AV. PARAL.LEL, 194",41.375116,2.152309,2,24,3
4,4,"C/ BILBAO, 174",41.409020,2.195415,23,9,4
...,...,...,...,...,...,...,...
501,501,"PG. MARITIM, 23 (HOSPITAL DEL MAR)",41.383365,2.194625,2,24,501
502,502,"C/GAVÀ, 81",41.370372,2.138944,9,7,502
503,503,"PG. MARíTIM, 31 B (ANNEXA A LA 12)",41.383475,2.194735,0,21,503
504,504,AV. LITORAL (PG MARÍTIM DEL PORT OLÍMPIC),41.388913,2.199311,4,23,504


In [35]:
#example queries

query = '''
    SELECT * 
    FROM bikes_table
    JOIN final_foursquare_table
    ON bikes_table."Bike Stop #" = final_foursquare_table."Bike Stop #"
    WHERE bikes_table.name IS NOT NULL;
'''  
result = pd.read_sql_query(query, conn)

result

,Unnamed: 0,name,latitude,longitude,free_bikes,empty_slots,Bike Stop #,Unnamed: 0,Bike Stop #,name,distance,venue_type
0,1,"PG. TORRAS I BAGES, 129",41.443365,2.190630,17,7,1,0,1,None,NaN,[]
1,2,"C/ SANT OLEGUER, 2",41.377532,2.170732,13,5,2,1,2,None,NaN,[]
2,3,"AV. PARAL.LEL, 194",41.375116,2.152309,2,24,3,2,3,None,NaN,[]
3,4,"C/ BILBAO, 174",41.409020,2.195415,23,9,4,3,4,Montjuic (Barcelona),181.0,Art Museum
4,5,"C/ D'ESCORNALBOU, 51",41.418079,2.176399,22,2,5,4,5,None,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
627,501,"PG. MARITIM, 23 (HOSPITAL DEL MAR)",41.383365,2.194625,2,24,501,627,501,La Fábrica del Sol,190.0,Museum
628,502,"C/GAVÀ, 81",41.370372,2.138944,9,7,502,628,502,None,NaN,[]
629,503,"PG. MARíTIM, 31 B (ANNEXA A LA 12)",41.383475,2.194735,0,21,503,629,503,None,NaN,[]
630,504,AV. LITORAL (PG MARÍTIM DEL PORT OLÍMPIC),41.388913,2.199311,4,23,504,630,504,None,NaN,[]


tables were join in SQL and compared to the original df in python. The total number of rows, total number of non-null rows, and some spot checks all matched. The data is validated as being reliable.